**Tensorflow Play**

Getting familiar with Tensorflow by developing a simple classifier.

In [1]:
import tensorflow as tf
from tensorflow import keras

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append("../src")
import utils as u

**Getting the Data**

In [2]:
# Load Data from Keras
X_train, X_test, X_valid, y_train, y_test, y_valid = u.get_fashion_mnist_data()

In [3]:
booleans_train = [True if x in [0,1,2,3,4,5,6,7] else False for x in y_train]
booleans_test = [True if x in [0,1,2,3,4,5,6,7] else False for x in y_test]
booleans_valid = [True if x in [0,1,2,3,4,5,6,7] else False for x in y_valid]

In [4]:
xtrain_a = X_train[booleans_train]
xtest_a = X_test[booleans_test]
xvalid_a = X_valid[booleans_valid]

ytrain_a = y_train[booleans_train]
ytest_a = y_test[booleans_test]
yvalid_a = y_valid[booleans_valid]

**Showcase Pretrained Models**

We have to train a model for labels `8` & `9`, but we only have a limited number of labels (200). We know that someone else, has already trained a similar task namely, classifies labels `0-7`. Given that some components could be reused, we are going to use that model.


**Model A**

First, we create an initial model that only classifies labels `0-7`. The idea is to then reuse this model to train labels `8` & `9`.


In [6]:
# Start with a simple sequential model
model = keras.models.Sequential()

# Flatten the input
model.add(keras.layers.Flatten(input_shape=[28, 28]))

# Add dense layers
model.add(keras.layers.Dense(300, activation="relu"))

# Add Final Layer
model.add(keras.layers.Dense(10, activation="softmax"))

# Add Optimizer & Compile model
optimizer = keras.optimizers.SGD(lr=0.3)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

checkpoint_cb = keras.callbacks.ModelCheckpoint("../models/classification_reusing_pretrained_layers_a.h5",
                                                save_best_only=True)

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,
                                                  restore_best_weights=True,
                                                  monitor='accuracy')

tensorboard_cb = keras.callbacks.TensorBoard(u.get_run_logdir())

In [7]:
%%time
model_train = model.fit(xtrain_a, ytrain_a, epochs=30, validation_data=(xvalid_a, yvalid_a),
                        callbacks=[checkpoint_cb,early_stopping_cb,tensorboard_cb])

Epoch 1/30
1375/1375 [==============================] - 3s 2ms/step - loss: 0.7900 - accuracy: 0.7039 - val_loss: 0.4356 - val_accuracy: 0.8304
Epoch 2/30
1375/1375 [==============================] - 2s 2ms/step - loss: 0.4554 - accuracy: 0.8242 - val_loss: 0.4154 - val_accuracy: 0.8444
Epoch 3/30
1375/1375 [==============================] - 2s 2ms/step - loss: 0.4063 - accuracy: 0.8454 - val_loss: 0.3898 - val_accuracy: 0.8511
Epoch 4/30
1375/1375 [==============================] - 2s 2ms/step - loss: 0.3829 - accuracy: 0.8563 - val_loss: 0.3534 - val_accuracy: 0.8661
Epoch 5/30
1375/1375 [==============================] - 2s 1ms/step - loss: 0.3588 - accuracy: 0.8651 - val_loss: 0.3830 - val_accuracy: 0.8536
Epoch 6/30
1375/1375 [==============================] - 2s 2ms/step - loss: 0.3449 - accuracy: 0.8682 - val_loss: 0.3716 - val_accuracy: 0.8641
Epoch 7/30
1375/1375 [==============================] - 2s 2ms/step - loss: 0.3311 - accuracy: 0.8762 - val_loss: 0.3809 - val_accuracy:

**Model B - Try 1**

In [8]:
booleans_train_b_8 = [True if x == 8 else False for x in y_train]
booleans_test_b_8 = [True if x == 8 else False for x in y_test]
booleans_valid_b_8 = [True if x == 8 else False for x in y_valid]

booleans_train_b_9 = [True if x == 9 else False for x in y_train]
booleans_test_b_9 = [True if x == 9 else False for x in y_test]
booleans_valid_b_9 = [True if x == 9 else False for x in y_valid]

In [9]:
xtrain_b_8 = X_train[booleans_train_b_8]
xtest_b_8 = X_test[booleans_test_b_8]
xvalid_b_8 = X_valid[booleans_valid_b_8]

ytrain_b_8 = y_train[booleans_train_b_8]
ytest_b_8 = y_test[booleans_test_b_8]
yvalid_b_8 = y_valid[booleans_valid_b_8]

In [10]:
xtrain_b_9 = X_train[booleans_train_b_9]
xtest_b_9 = X_test[booleans_test_b_9]
xvalid_b_9 = X_valid[booleans_valid_b_9]

ytrain_b_9 = y_train[booleans_train_b_9]
ytest_b_9 = y_test[booleans_test_b_9]
yvalid_b_9 = y_valid[booleans_valid_b_9]

In [11]:
xtrain_b = np.concatenate([xtrain_b_8[0:100],xtrain_b_9[0:100]])
xtest_b = np.concatenate([xtest_b_8,xtest_b_9])
xvalid_b = np.concatenate([xvalid_b_8[0:100],xvalid_b_9[0:100]])

ytrain_b = np.concatenate([ytrain_b_8[0:100],ytrain_b_9[0:100]])
ytest_b = np.concatenate([ytest_b_8,ytest_b_9])
yvalid_b = np.concatenate([yvalid_b_8[0:100],yvalid_b_9[0:100]])

In [12]:
ytrain_b = np.array([1 if x == 8 else 0 for x in ytrain_b])
ytest_b = np.array([1 if x == 8 else 0 for x in ytest_b])
yvalid_b = np.array([1 if x == 8 else 0 for x in yvalid_b])

In [13]:
# Start with a simple sequential model
model = keras.models.Sequential()

# Flatten the input
model.add(keras.layers.Flatten(input_shape=[28, 28]))

# Add dense layers
model.add(keras.layers.Dense(300, activation="relu"))

# Add Final Layer
model.add(keras.layers.Dense(2, activation="softmax"))

# Add Optimizer & Compile model
optimizer = keras.optimizers.SGD(lr=0.3)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

checkpoint_cb = keras.callbacks.ModelCheckpoint("../models/classification_reusing_pretrained_layers_b_a.h5",
                                                save_best_only=True)

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,
                                                  restore_best_weights=True,
                                                  monitor='accuracy')

tensorboard_cb = keras.callbacks.TensorBoard(u.get_run_logdir())

In [14]:
%%time
model_train = model.fit(xtrain_b, ytrain_b, epochs=30, validation_data=(xvalid_b, yvalid_b),
                        callbacks=[checkpoint_cb,early_stopping_cb,tensorboard_cb])

Epoch 1/30
7/7 [==============================] - 2s 237ms/step - loss: 2.6424 - accuracy: 0.5540 - val_loss: 0.5435 - val_accuracy: 0.6700
Epoch 2/30
7/7 [==============================] - 0s 14ms/step - loss: 0.3085 - accuracy: 0.8666 - val_loss: 3.5729 - val_accuracy: 0.5000
Epoch 3/30
7/7 [==============================] - 0s 11ms/step - loss: 1.6036 - accuracy: 0.5994 - val_loss: 0.3575 - val_accuracy: 0.9850
Epoch 4/30
7/7 [==============================] - 0s 14ms/step - loss: 1.0325 - accuracy: 0.7996 - val_loss: 0.5093 - val_accuracy: 0.9700
Epoch 5/30
7/7 [==============================] - 0s 13ms/step - loss: 0.8035 - accuracy: 0.9205 - val_loss: 0.6678 - val_accuracy: 0.5000
Epoch 6/30
7/7 [==============================] - 0s 12ms/step - loss: 0.6035 - accuracy: 0.6855 - val_loss: 0.5785 - val_accuracy: 0.8250
Epoch 7/30
7/7 [==============================] - 0s 13ms/step - loss: 0.5221 - accuracy: 0.8975 - val_loss: 0.2556 - val_accuracy: 0.9900
Epoch 8/30
7/7 [==========

In [15]:
model_train.model.evaluate(xtest_b, ytest_b)

63/63 [==============================] - 0s 2ms/step - loss: 3.5936 - accuracy: 0.5000


[3.5936391353607178, 0.5]

**Applying Transfer Learning**

When you train `model_B_on_A`, it will also affect `model_A`. If you want to avoid that, you need to clone `model_A` before you reuse its layers. To do this, you clone `model A`’s architecture with `clone_model()`, then copy its weights (since `clone_model()` does not clone the weights)

In [17]:
model_A = keras.models.load_model("../models/classification_reusing_pretrained_layers_a.h5")
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

Now you could train model_B_on_A for task B, but since the new output layer was initialized randomly it will make large errors (at least during the first few epochs), so there will be large error gradients that may wreck the reused weights. To avoid this, one approach is to freeze the reused layers during the first few epochs, giving the new layer some time to learn reasonable weights. To do this, set every layer’s trainable attribute to False and compile the model:

In [23]:
model_B_on_A = keras.models.Sequential(model_A_clone.layers[:-1])
model_B_on_A.add(keras.layers.Dense(2, activation="softmax"))

In [24]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

You must always compile your model after you freeze or unfreeze layers.

In [25]:
model_B_on_A.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer,
                     metrics=["accuracy"])

Now you can train the model for a few epochs, then unfreeze the reused layers (which requires compiling the model again) and continue training to fine-tune the reused layers for task B

In [26]:
history = model_B_on_A.fit(xtrain_b, ytrain_b, epochs=4,
                           validation_data=(xvalid_b, yvalid_b))

Epoch 1/4
7/7 [==============================] - 1s 46ms/step - loss: 0.3618 - accuracy: 0.7703 - val_loss: 0.0629 - val_accuracy: 0.9900
Epoch 2/4
7/7 [==============================] - 0s 10ms/step - loss: 0.0773 - accuracy: 0.9912 - val_loss: 0.0455 - val_accuracy: 0.9950
Epoch 3/4
7/7 [==============================] - 0s 9ms/step - loss: 0.0489 - accuracy: 0.9924 - val_loss: 0.0411 - val_accuracy: 0.9950
Epoch 4/4
7/7 [==============================] - 0s 9ms/step - loss: 0.0741 - accuracy: 0.9892 - val_loss: 0.0302 - val_accuracy: 0.9950


In [27]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

In [28]:
model_B_on_A.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
                     metrics=["accuracy"])

checkpoint_cb = keras.callbacks.ModelCheckpoint("../models/classification_reusing_pretrained_layers_b_b.h5",
                                                save_best_only=True)

history = model_B_on_A.fit(xtrain_b, ytrain_b, epochs=16,
                           validation_data=(xvalid_b, yvalid_b),
                           callbacks=[checkpoint_cb,early_stopping_cb,tensorboard_cb])

Epoch 1/16
7/7 [==============================] - 2s 315ms/step - loss: 0.0736 - accuracy: 0.9892 - val_loss: 0.0177 - val_accuracy: 0.9950
Epoch 2/16
7/7 [==============================] - 0s 10ms/step - loss: 0.0438 - accuracy: 0.9914 - val_loss: 0.0173 - val_accuracy: 0.9950
Epoch 3/16
7/7 [==============================] - 0s 8ms/step - loss: 0.0320 - accuracy: 0.9950 - val_loss: 0.0120 - val_accuracy: 0.9950
Epoch 4/16
7/7 [==============================] - 0s 10ms/step - loss: 0.0340 - accuracy: 0.9881 - val_loss: 0.0139 - val_accuracy: 0.9950
Epoch 5/16
7/7 [==============================] - 0s 10ms/step - loss: 0.0116 - accuracy: 0.9981 - val_loss: 0.0161 - val_accuracy: 1.0000
Epoch 6/16
7/7 [==============================] - 0s 10ms/step - loss: 0.0072 - accuracy: 1.0000 - val_loss: 0.0096 - val_accuracy: 1.0000
Epoch 7/16
7/7 [==============================] - 0s 8ms/step - loss: 0.0050 - accuracy: 1.0000 - val_loss: 0.0094 - val_accuracy: 1.0000
Epoch 8/16
7/7 [============

In [29]:
history.model.evaluate(xtest_b, ytest_b)

63/63 [==============================] - 0s 1ms/step - loss: 0.0201 - accuracy: 0.9965


[0.020140551030635834, 0.9965000152587891]